# DeepLabV3 Studies

TBD

# References

This section has references to papers and code that I have used to put together this demo. Some of the code comes directly from other open source repos and in some cases I have modified it.  I have tried to give credit to all of the sources but if you see something that needs to be added, just let me know

* Google's DeepLabV3 repository. https://github.com/tensorflow/models/tree/master/research/deeplab
* Extract a mask from DeepLabV3 segmentation. https://stackoverflow.com/questions/52113864/extract-image-segmentation-map-from-tensorflow-deeplab-v3-demo
* Transfer learning using DeepLabV3. https://expoundai.wordpress.com/2019/08/30/transfer-learning-for-segmentation-using-deeplabv3-in-pytorch/
ipython widgets. https://ipywidgets.readthedocs.io/en/latest/
* Google Colab off the shelf notebook to test DeepLabV3. https://github.com/tensorflow/models/blob/master/research/deeplab/deeplab_demo.ipynb
* Python `tempfile` module. https://docs.python.org/2/library/tempfile.html
* Python `urlib` module. https://docs.python.org/2/library/urllib.html

# Requirements

The software requirements for this notebook to run (Python packages) are in the `dlv3tf` conda environment. Some specific things:

* This notebook uses Tensor Flow 1.x, more specifically 1.15. Tensor Flow 2.0 is released, but this notebook (and Google's current (main branch) DeepLabV3 code only works with TF 1.15.

# Environment

In [11]:
import os
import sys
import tarfile
import urllib

from IPython import display
from ipywidgets import interact
from ipywidgets import interactive
from matplotlib import gridspec

from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf

In [2]:
print("Using TensorFlow Version:", tf.__version__)

Using TensorFlow Version: 1.15.0


The sample page for the DeepLabV3 has a Colab notebook that has a class to implement the model and run inference. At this time I am going to just re-use the code from the Google page here. Most everybody that I have seen simply re-uses this code, there is really nothing more than I can add.  

I may, however simplity it or slightly adapt it for my configuration.  In any case much of the code in the next section (`class DeepLab` model comes from https://github.com/tensorflow/models/blob/master/research/deeplab/deeplab_demo.ipynb)

In [12]:
class DeepLabModel(object):
  """Class to load deeplab model and run inference."""

  INPUT_TENSOR_NAME = 'ImageTensor:0'
  OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
  INPUT_SIZE = 513
  FROZEN_GRAPH_NAME = 'frozen_inference_graph'

  def __init__(self, tarball_path):
    """Creates and loads pretrained deeplab model."""
    self.graph = tf.Graph()

    graph_def = None
    # Extract frozen graph from tar archive.
    tar_file = tarfile.open(tarball_path)
    for tar_info in tar_file.getmembers():
      if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
        file_handle = tar_file.extractfile(tar_info)
        graph_def = tf.GraphDef.FromString(file_handle.read())
        break

    tar_file.close()

    if graph_def is None:
      raise RuntimeError('Cannot find inference graph in tar archive.')

    with self.graph.as_default():
      tf.import_graph_def(graph_def, name='')

    self.sess = tf.Session(graph=self.graph)

  def run(self, image):
    """Runs inference on a single image.

    Args:
      image: A PIL.Image object, raw input image.

    Returns:
      resized_image: RGB image resized from original input image.
      seg_map: Segmentation map of `resized_image`.
    """
    width, height = image.size
    resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
    target_size = (int(resize_ratio * width), int(resize_ratio * height))
    resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
    batch_seg_map = self.sess.run(
        self.OUTPUT_TENSOR_NAME,
        feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
    seg_map = batch_seg_map[0]
    return resized_image, seg_map

For this notebook, I will use (first) the Xception backend. It will have to be downloaded. The Colab notebook from Google has some code that allows you to choose a pre-trained model. Since at some point I will want to try other models, I will use use that code

In [13]:
_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
_MODEL_URLS = {
    'mobilenetv2_coco_voctrainaug':
        'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
    'mobilenetv2_coco_voctrainval':
        'deeplabv3_mnv2_pascal_trainval_2018_01_29.tar.gz',
    'xception_coco_voctrainaug':
        'deeplabv3_pascal_train_aug_2018_01_04.tar.gz',
    'xception_coco_voctrainval':
        'deeplabv3_pascal_trainval_2018_01_04.tar.gz',
}
_TARBALL_NAME = 'deeplab_model.tar.gz'

Set the next variable to the name of the model that we want to run. 

In [14]:
MODEL_NAME = 'mobilenetv2_coco_voctrainaug'

Where we want to store the downloaded model. The Google notebook uses a directory in `\tmp`

In [15]:
MODEL_DIR = 'tfmodels/'
model_dir = MODEL_DIR

In [16]:
download_path = os.path.join(model_dir, _TARBALL_NAME)
print("Model will be downloaded to:", download_path)

Model will be downloaded to: tfmodels/deeplab_model.tar.gz


In [17]:
print(_MODEL_URLS[MODEL_NAME])

deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz


In [18]:
print('downloading model, this might take a while...')
urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + _MODEL_URLS[MODEL_NAME],
                   download_path)
print('download completed! loading DeepLab model...')

downloading model, this might take a while...
download completed! loading DeepLab model...


# Test Loaded Model

Download the model. It gets downloaded to a `tmp` directory in `\tmp\<generated temp file name>`

In [19]:
MODEL = DeepLabModel(download_path)
print('model loaded successfully!')

model loaded successfully!
